In [1]:
import numpy as np
import pandas as pd # for loading data

In [2]:
cd ..

/home/nicktehrany/Documents/Uni/3rd Year/P4/Machine Learning/Project


In [3]:
# Importing our own Functions
from util.helperfunctions import one_hot, one_hot_genres, clean_text, one_hot_reverse

/home/nicktehrany/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/nicktehrany/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/nicktehrany/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/nicktehrany/.local/lib/python3.6/site-packages/tensorflow/python/frame

In [4]:
cd src

/home/nicktehrany/Documents/Uni/3rd Year/P4/Machine Learning/Project/src


In [5]:
train_df = pd.read_csv("train.csv")

## Cleaning the test set lyrics and creating One-Hots for train data's genres and just cleaned lyrics

In [6]:
lyrics, word_list = clean_text(train_df['Lyrics'], 1)

# writing cleaned text back to df
index = 0
for text in lyrics:
    train_df['Lyrics'][index] = text
    index+=1
del lyrics

In [7]:
train_genres = one_hot_genres(train_df['Genre'])

## Replace all the lyrics with their respctive vector of word occurrences to be used as the input layer for the neural network

In [8]:
train_lyrics = np.zeros(shape=(len(train_df),len(word_list)))
for index, value in train_df.iterrows():
    train_lyrics[index] = one_hot(str(train_df['Lyrics'][index]), word_list, len(word_list))
print(train_lyrics.shape)
lyrics_size = len(word_list)

# Cleaning no more used variables
del train_df

(12240, 37818)


## Cleaning the test set lyrics and creating One-Hots for test data's genres and just cleaned lyrics

In [9]:
test_df = pd.read_csv("test.csv")
lyrics, dummy_list = clean_text(test_df['Lyrics'], 0)

# writing cleaned text back to df
index = 0
for text in lyrics:
    test_df['Lyrics'][index] = text
    index+=1
del lyrics

In [10]:
test_genres = one_hot_genres(test_df['Genre'])

In [11]:
test_lyrics = np.zeros(shape=(len(test_df), lyrics_size))
for index, value in test_df.iterrows():
    test_lyrics[index] = one_hot(str(test_df['Lyrics'][index]), word_list, lyrics_size)
print(test_lyrics.shape)

# Cleaning no more used variables
del test_df

(2160, 37818)


# Neural Network stuff 
## **Modify and run only the cells below to change the neural network**

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout

model = Sequential()
model.add(Dense(units=256, input_shape=(lyrics_size,)))
model.add(Activation('relu'))            # activation layer
model.add(Dropout(0.8))
model.add(Dense(32))
model.add(Dense(8))
model.add(Activation('softmax'))         # output class probabilities

model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               9681664   
_________________________________________________________________
activation (Activation)      (None, 256)               0         
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                8224      
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 264       
_________________________________________________________________
activation_1 (Activation)    (None, 8)                 0         
Tota

In [13]:
from tensorflow.keras.optimizers import SGD, Adam

optimizer = Adam(lr=0.0001) # lr is the learning rate
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
model.fit(train_lyrics, train_genres, epochs=7, validation_split=17/100, batch_size=16);

Train on 10159 samples, validate on 2081 samples
Epoch 1/7
10159/10159 [==============================] - 16s 2ms/sample - loss: 1.8922 - acc: 0.2853 - val_loss: 1.6814 - val_acc: 0.4306
Epoch 2/7
10159/10159 [==============================] - 16s 2ms/sample - loss: 1.5472 - acc: 0.4597 - val_loss: 1.4894 - val_acc: 0.4988
Epoch 3/7
10159/10159 [==============================] - 16s 2ms/sample - loss: 1.3333 - acc: 0.5408 - val_loss: 1.3959 - val_acc: 0.5195
Epoch 4/7
10159/10159 [==============================] - 16s 2ms/sample - loss: 1.1676 - acc: 0.6087 - val_loss: 1.3439 - val_acc: 0.5247
Epoch 5/7
10159/10159 [==============================] - 16s 2ms/sample - loss: 1.0475 - acc: 0.6532 - val_loss: 1.3198 - val_acc: 0.5286
Epoch 6/7
10159/10159 [==============================] - 16s 2ms/sample - loss: 0.9468 - acc: 0.6892 - val_loss: 1.3035 - val_acc: 0.5358
Epoch 7/7
10159/10159 [==============================] - 16s 2ms/sample - loss: 0.8454 - acc: 0.7260 - val_loss: 1.3016 - v

## Testing the Neural Network's performace + Confusion Matrix

In [15]:
results = model.evaluate(test_lyrics, test_genres, batch_size=16)

2160/2160 [==============================] - 1s 279us/sample - loss: 1.3178 - acc: 0.5296


In [16]:
from sklearn.metrics import confusion_matrix

y_pred=model.predict_classes(test_lyrics)
cm=confusion_matrix(one_hot_reverse(test_genres),y_pred)
print(cm)

[[ 89  33  14  35  41  33  24  23]
 [ 27  85   0  27  15  55  35  33]
 [  6   1 219   1  27   3   5   0]
 [  7  11   0 193  14   6  17  39]
 [ 12   8  21  13 145   9  19  10]
 [  7  29   3   9   6 166  20  21]
 [ 18  20  21  13  45  21 101  24]
 [ 18  27   0  40   7  21  22 146]]


In [17]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

genre_list = ['Pop','Rock', 'Hip-Hop/Rap', 'Country', 'R&B/Soul', 'Metal', 'Alternative/Indie', 'Folk']

df_cm = pd.DataFrame(cm, index = genre_list, columns = genre_list)
print(df_cm)
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True, linewidth=0.5, fmt='g', cmap="BuPu")
b, t = plt.ylim() # discover the values for bottom and top
b += 0.5 # Add 0.5 to the bottom
t -= 0.5 # Subtract 0.5 from the top
plt.ylim(b, t) # update the ylim(bottom, top) values

                   Pop  Rock  Hip-Hop/Rap  Country  R&B/Soul  Metal  \
Pop                 89    33           14       35        41     33   
Rock                27    85            0       27        15     55   
Hip-Hop/Rap          6     1          219        1        27      3   
Country              7    11            0      193        14      6   
R&B/Soul            12     8           21       13       145      9   
Metal                7    29            3        9         6    166   
Alternative/Indie   18    20           21       13        45     21   
Folk                18    27            0       40         7     21   

                   Alternative/Indie  Folk  
Pop                               24    23  
Rock                              35    33  
Hip-Hop/Rap                        5     0  
Country                           17    39  
R&B/Soul                          19    10  
Metal                             20    21  
Alternative/Indie                101    24  


(8.0, 0.0)

## **Don't rerun the cell below, as this will result in adding dimensions to the sets, which will not work! Can rerun all other cells except this one.**

In [18]:
cnn_lyrics_train = train_lyrics[:, :, None]
cnn_lyrics_test = test_lyrics[:, :, None]

## Convolutional Neural Network

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv1D, MaxPool2D, Dropout, Flatten

model = Sequential()
model.add(Conv1D(128, kernel_size=(1), activation='relu', input_shape=(len(word_list),1)))
model.add(Conv1D(32, (1), activation='relu'))
model.add(Dropout(0.8)) # Dropout 25% of the nodes of the previous layer during training
model.add(Flatten())     # Flatten, and add a fully connected layer
model.add(Dense(16, activation='relu')) 
model.add(Dropout(0.2))
model.add(Dense(8, activation='softmax')) # Last layer: 10 class nodes, with dropout
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_2 (Conv1D)            (None, 37818, 128)        256       
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 37818, 32)         4128      
_________________________________________________________________
dropout_3 (Dropout)          (None, 37818, 32)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1210176)           0         
_________________________________________________________________
dense_5 (Dense)              (None, 16)                19362832  
_________________________________________________________________
dropout_4 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 8)                

In [22]:
from tensorflow.keras.optimizers import Adam

optimizer = Adam()
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(cnn_lyrics_train, train_genres, epochs=5, batch_size=16, validation_split=17/100)

Train on 10159 samples, validate on 2081 samples
Epoch 1/5
10159/10159 [==============================] - 352s 35ms/sample - loss: 1.8404 - acc: 0.3008 - val_loss: 1.6148 - val_acc: 0.4402
Epoch 2/5
10159/10159 [==============================] - 353s 35ms/sample - loss: 1.5647 - acc: 0.4019 - val_loss: 1.4716 - val_acc: 0.4733
Epoch 3/5
10159/10159 [==============================] - 374s 37ms/sample - loss: 1.4043 - acc: 0.4517 - val_loss: 1.4236 - val_acc: 0.4810
Epoch 4/5
10159/10159 [==============================] - 388s 38ms/sample - loss: 1.2965 - acc: 0.4913 - val_loss: 1.4237 - val_acc: 0.4762
Epoch 5/5
10159/10159 [==============================] - 370s 36ms/sample - loss: 1.2234 - acc: 0.5125 - val_loss: 1.4142 - val_acc: 0.4825


In [23]:
results = model.evaluate(cnn_lyrics_test, test_genres, batch_size=16)

2160/2160 [==============================] - 18s 9ms/sample - loss: 1.4211 - acc: 0.4796
